In [1]:
import pandas as pd
#from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from gensim.models import word2vec
import pickle
import nltk.data
import os
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [4]:
# Read data from files 
train = pd.read_csv("../data/train.csv", header=0)
test = pd.read_csv( "../data/test.csv", header=0)

# Verify the number of comments that were read
print("Read %d labeled train reviews and  %d unlabelled test reviews" % (len(train),len(test)))
train_comments = train['comment_text'].fillna("_na_").tolist()
test_comments = test['comment_text'].fillna("_na_").tolist()
all_comments = train['comment_text'].fillna("_na_").tolist() + test['comment_text'].fillna("_na_").tolist() 

with open("data/raw_comments/all_comments.csv", "w+") as comments_file:
    i=0
    for comment in all_comments:
        comment = re.sub("[^a-zA-Z]"," ",str(comment))
        comments_file.write("%s\n" % comment)
        
with open("data/raw_comments/train_comments.csv", "w+") as comments_file:
    i=0
    for comment in train_comments:
        comment = re.sub("[^a-zA-Z]"," ",str(comment))
        comments_file.write("%s\n" % comment)
        
with open("data/raw_comments/test_comments.csv", "w+") as comments_file:
    i=0
    for comment in test_comments:
        comment = re.sub("[^a-zA-Z]"," ",str(comment))
        comments_file.write("%s\n" % comment)

FileNotFoundError: File b'../data/train.csv' does not exist

##### 1. Preprocess Text Reviews For Creating Word Vectors

The <b>sentences</b> iterable can be simply a list, but for larger corpora, consider a generator that streams the sentences directly from disk/network, without storing everything in RAM. See BrownCorpus, Text8Corpus or LineSentence in the gensim.models.word2vec module for such examples.

<b>min_count</b> ignore all words and bigrams with total collected count lower than this.

<b>threshold</b> represents a score threshold for forming the phrases (higher means fewer phrases). A phrase of words a followed by b is accepted if the score of the phrase is greater than threshold. see the scoring setting.

<b>max_vocab_size</b> is the maximum size of the vocabulary. Used to control pruning of less common words, to keep memory under control. The default of 40M needs about 3.6GB of RAM; increase/decrease max_vocab_size depending on how much available memory you have.

<b>delimiter</b> is the glue character used to join collocation tokens, and should be a byte string (e.g. b’_’).

<b>scoring</b> specifies how potential phrases are scored for comparison to the threshold setting. scoring can be set with either a string that refers to a built-in scoring function, or with a function with the expected parameter names. Two built-in scoring functions are available by setting scoring to a string:

‘default’: from “Efficient Estimaton of Word Representations in Vector Space” by
Mikolov, et. al.: (count(worda followed by wordb) - min_count) * N / (count(worda) * count(wordb)) > threshold`, where <b>N</b> is the total vocabulary size.
<b>npmi</b>: normalized pointwise mutual information, from “Normalized (Pointwise) Mutual
Information in Colocation Extraction” by Gerlof Bouma: ln(prop(worda followed by wordb) / (prop(worda)*prop(wordb))) / - ln(prop(worda followed by wordb) where prop(n) is the count of n / the count of everything in the entire corpus.
‘npmi’ is more robust when dealing with common words that form part of common bigrams, and ranges from -1 to 1, but is slower to calculate than the default.

To use a custom scoring function, create a function with the following parameters and set the scoring parameter to the custom function. You must use all the parameters in your function call, even if the function does not require all the parameters.

<b>worda_count</b>: number of occurrances in sentences of the first token in the phrase being scored wordb_count: number of occurrances in sentences of the second token in the phrase being scored bigram_count: number of occurrances in sentences of the phrase being scored len_vocab: the number of unique tokens in sentences min_count: the min_count setting of the Phrases class corpus_word_count: the total number of (non-unique) tokens in sentences
A scoring function without any of these parameters (even if the parameters are not used) will raise a ValueError on initialization of the Phrases class. The scoring function must be picklable.

common_terms is an optionnal list of “stop words” that won’t affect frequency count of expressions containing them.

In [21]:
class FileToComments(object):    
    def __init__(self, filename):
        self.filename = filename
        self.stop = set(nltk.corpus.stopwords.words('english'))
        
    def __iter__(self):
        
        def comment_to_wordlist(comment, remove_stopwords=True):
            comment = str(comment)
            words = comment.lower().split()
            #if remove_stopwords:
            #    stops = set(stopwords.words("english"))
            #    words = [w for w in words if not w in stops]
            return(words)
    
        for line in open(self.filename, 'r'):
            #line = unicode(line, 'utf-8')
            tokenized_comment = comment_to_wordlist(line, tokenizer)
            yield tokenized_comment
        
all_comments = FileToComments('data/raw_comments/all_comments.csv')
train_comments = FileToComments('data/raw_comments/train_comments.csv')
test_comments = FileToComments('data/raw_comments/test_comments.csv')

In [22]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
# Train Tokenizer on all comments
bigram = Phrases(all_comments, min_count=30, threshold=15)
bigram_phraser = Phraser(bigram) 

2017-12-29 12:21:56,677 : INFO : collecting all words and their counts
2017-12-29 12:21:56,679 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2017-12-29 12:21:59,229 : INFO : PROGRESS: at sentence #10000, processed 676828 words and 287566 word types
2017-12-29 12:22:01,831 : INFO : PROGRESS: at sentence #20000, processed 1351623 words and 489953 word types
2017-12-29 12:22:04,331 : INFO : PROGRESS: at sentence #30000, processed 2029376 words and 665776 word types
2017-12-29 12:22:07,023 : INFO : PROGRESS: at sentence #40000, processed 2728907 words and 830315 word types
2017-12-29 12:22:09,500 : INFO : PROGRESS: at sentence #50000, processed 3397508 words and 977689 word types
2017-12-29 12:22:12,016 : INFO : PROGRESS: at sentence #60000, processed 4076917 words and 1119668 word types
2017-12-29 12:22:14,524 : INFO : PROGRESS: at sentence #70000, processed 4757328 words and 1255084 word types
2017-12-29 12:22:17,144 : INFO : PROGRESS: at sentence #80000, processe

In [23]:
train_tokens = [bigram_phraser[comment] for comment in train_comments]
test_tokens = [bigram_phraser[comment] for comment in test_comments]
all_tokens = [bigram_phraser[comment] for comment in all_comments]

stops = set(stopwords.words("english"))

clean_train_tokens = []
for token in train_tokens:
    words = [w for w in token if not w in stops]
    clean_train_tokens += [words]
    
clean_test_tokens = []
for token in test_tokens:
    words = [w for w in token if not w in stops]
    clean_test_tokens += [words]

clean_all_tokens = []
for token in all_tokens:
    words = [w for w in token if not w in stops]
    clean_all_tokens += [words]
print('tokens cleaned')

tokens cleaned


In [24]:
#Pickle the tokens file for further use
import pickle
with open('data/tokenized_comments/tokenized_test_comments.pickle', 'wb') as filename:
    pickle.dump(clean_test_tokens, filename, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('data/tokenized_comments/tokenized_train_comments.pickle', 'wb') as filename:
    pickle.dump(clean_train_tokens, filename, protocol=pickle.HIGHEST_PROTOCOL)

with open('data/tokenized_comments/tokenized_all_comments.pickle', 'wb') as filename:
    pickle.dump(clean_all_tokens, filename, protocol=pickle.HIGHEST_PROTOCOL)
print('files saved to pickle data/tokenized_comments/...')

files saved to pickle data/tokenized_comments/...


##### 2. Training and Saving Your Model

With the list of nicely parsed sentences, we're ready to train the model. There are a number of parameter choices that affect the run time and the quality of the final model that is produced. For details on the algorithms below, see the word2vec API documentation as well as the Google documentation. 

 - Architecture: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results. 
 - Training algorithm: Hierarchical softmax (default) or negative sampling. For us, the default worked well.
 - Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.
 - Word vector dimensionality: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300. 
 - Context / window size: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).
 - Worker threads: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.
 - Minimum word count: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

In [25]:
#Load Pre-saved tokenized comments
with open('data/tokenized_comments/tokenized_all_comments.pickle', 'rb') as filename:
    all_comments = pickle.load(filename)
    
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)


# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 20   # Minimum word count                        
num_workers = 16       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
print("Training model...")
model = word2vec.Word2Vec(all_comments,
                          workers=num_workers,
                          size=num_features,
                          min_count = min_word_count,
                          window = context,
                          sample = downsampling
                         )

# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)
model_name = "models/%sfeatures_%sminwords_%scontext" % (num_features,min_word_count,context)
model.save(model_name)

2017-12-29 12:31:15,777 : INFO : collecting all words and their counts
2017-12-29 12:31:15,778 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-29 12:31:15,884 : INFO : PROGRESS: at sentence #10000, processed 332303 words, keeping 35481 word types


Training model...


2017-12-29 12:31:15,988 : INFO : PROGRESS: at sentence #20000, processed 664751 words, keeping 52404 word types
2017-12-29 12:31:16,097 : INFO : PROGRESS: at sentence #30000, processed 998240 words, keeping 66195 word types
2017-12-29 12:31:16,205 : INFO : PROGRESS: at sentence #40000, processed 1342842 words, keeping 78285 word types
2017-12-29 12:31:16,310 : INFO : PROGRESS: at sentence #50000, processed 1672684 words, keeping 89015 word types
2017-12-29 12:31:16,412 : INFO : PROGRESS: at sentence #60000, processed 2007133 words, keeping 98715 word types
2017-12-29 12:31:16,515 : INFO : PROGRESS: at sentence #70000, processed 2341001 words, keeping 107515 word types
2017-12-29 12:31:16,620 : INFO : PROGRESS: at sentence #80000, processed 2684149 words, keeping 116238 word types
2017-12-29 12:31:16,723 : INFO : PROGRESS: at sentence #90000, processed 3020627 words, keeping 124056 word types
2017-12-29 12:31:16,838 : INFO : PROGRESS: at sentence #100000, processed 3398527 words, keepin

2017-12-29 12:31:58,762 : INFO : PROGRESS: at 45.62% examples, 636825 words/s, in_qsize 31, out_qsize 0
2017-12-29 12:31:59,784 : INFO : PROGRESS: at 46.76% examples, 636224 words/s, in_qsize 31, out_qsize 0
2017-12-29 12:32:00,786 : INFO : PROGRESS: at 47.99% examples, 637385 words/s, in_qsize 31, out_qsize 0
2017-12-29 12:32:01,805 : INFO : PROGRESS: at 49.21% examples, 637537 words/s, in_qsize 31, out_qsize 0
2017-12-29 12:32:02,809 : INFO : PROGRESS: at 50.41% examples, 637494 words/s, in_qsize 30, out_qsize 1
2017-12-29 12:32:03,818 : INFO : PROGRESS: at 51.60% examples, 637980 words/s, in_qsize 30, out_qsize 1
2017-12-29 12:32:04,824 : INFO : PROGRESS: at 52.76% examples, 637733 words/s, in_qsize 31, out_qsize 0
2017-12-29 12:32:05,831 : INFO : PROGRESS: at 54.00% examples, 638108 words/s, in_qsize 31, out_qsize 0
2017-12-29 12:32:06,846 : INFO : PROGRESS: at 55.12% examples, 637213 words/s, in_qsize 29, out_qsize 2
2017-12-29 12:32:07,893 : INFO : PROGRESS: at 56.24% examples, 6